# Activity: Validate and clean your data

## Introduction

In this activity, you will use input validation and label encoding to prepare a dataset for analysis. These are fundamental techniques used in all types of data analysis, from simple linear regression to complex neural networks. 

In this activity, you are a data professional an investment firm that is attempting to invest in private companies with a valuation of at least $1 billion. These are often known as "unicorns." Your client wants to develop a better understanding of unicorns, with the hope they can be early investors in future highly successful companies. They are particularly interested in the investment strategies of the three top unicorn investors: Sequoia Capital, Tiger Global Management, and Accel. 

In [ ]:
## Step 1: Imports

Import relevant Python libraries and packages: `numpy`, `pandas`, `seaborn`, and `pyplot` from `matplotlib`.

In [ ]:
# Import libraries and packages.

### YOUR CODE HERE ### 


### Load the dataset

The data contains details about unicorn companies, such as when they were founded, when they achieved unicorn status, and their current valuation. The dataset `Modified_Unicorn_Companies.csv` is loaded as `companies`, now display the first five rows. The variables in the dataset have been adjusted to suit the objectives of this lab, so they may be different from similar data used in prior labs. As shown in this cell, the dataset has been automatically loaded in for you. You do not need to download the .csv file, or provide more code, in order to access the dataset and proceed with this lab. Please continue with this activity by completing the following instructions.

In [ ]:
# Run this cell so pandas displays all columns
pd.set_option('display.max_columns', None)

In [ ]:
# RUN THIS CELL TO IMPORT YOUR DATA. 
companies = pd.read_csv(r'C:\Users\saswa\Documents\GitHub\Python-For-Data-Analysis\Course-3\Data\Module_3\Modified_Unicorn_Companies.csv')

# Display the first five rows.
### YOUR CODE HERE ### 


## Step 2: Data cleaning


Begin by displaying the data types of the columns in `companies`.

In [ ]:
# Display the data types of the columns.

### YOUR CODE HERE ###


### Modify the data types

Notice that the data type of the `Date Joined` column is an `object`&mdash;in this case, a string. Convert this column to `datetime` to make it more usable. 

In [ ]:
# Apply necessary datatype conversions.

### YOUR CODE HERE ###


### Create a new column

Add a column called `Years To Unicorn`, which is the number of years between when the company was founded and when it became a unicorn.

In [ ]:
# Create the column Years To Unicorn.

### YOUR CODE HERE ###


**QUESTION: Why might your client be interested in how quickly a company achieved unicorn status?**

[Write your response here. Double-click (or enter) to edit.]

### Input validation

The data has some issues with bad data, duplicate rows, and inconsistent `Industry` labels.

Identify and correct each of these issues.

#### Correcting bad data

Get descriptive statistics for the `Years To Unicorn` column.

In [ ]:
# Identify and correct the issue with Years To Unicorn.

### YOUR CODE HERE ###


Isolate all rows where the `Years To Unicorn` column contains a negative value.

In [ ]:
# Isolate any rows where `Years To Unicorn` is negative

### YOUR CODE HERE ###

**Question: How many rows have negative values in the `Years To Unicorn` column, and what companies are they for?**


[Write your response here. Double-click (or enter) to edit.]

An internet search reveals that InVision was founded in 2011. Replace the value at `Year Founded` with `2011` for InVision's row. 

In [ ]:
# Replace InVision's `Year Founded` value with 2011

### YOUR CODE HERE ###

# Verify the change was made properly

### YOUR CODE HERE ###


Now, recalculate all the values in the `Years To Unicorn` column to remove the negative value for InVision. Verify that there are no more negative values afterwards.

In [ ]:
# Recalculate all values in the `Years To Unicorn` column

### YOUR CODE HERE ###

# Verify that there are no more negative values in the column

### YOUR CODE HERE ###


#### Issues with `Industry` labels

The company provided you with the following list of industry labels to identify in the data for `Industry`. 

**Note:** Any labels in the `Industry` column that are not in `industry_list` are misspellings.

In [ ]:
# List provided by the company of the expected industry labels in the data
industry_list = ['Artificial intelligence', 'Other','E-commerce & direct-to-consumer', 'Fintech',\
       'Internet software & services','Supply chain, logistics, & delivery', 'Consumer & retail',\
       'Data management & analytics', 'Edtech', 'Health', 'Hardware','Auto & transportation', \
        'Travel', 'Cybersecurity','Mobile & telecommunications']

First, check if there are values in the `Industry` column that are not in `industry_list`. If so, what are they?

In [ ]:
# Check which values are in `Industry` but not in `industry_list`

### YOUR CODE HERE ###


**Question: Which values currently exist in the `Industry` column that are not in `industry_list`?**


[Write your response here. Double-click (or enter) to edit.]

Now, correct the bad entries in the `Industry` column by replacing them with an approved string from `industry_list`. To do this, use the [`replace()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html) `Series` method on the `Industry` series. When you pass a dictionary to the method, it will replace the data in the series where that data matches the dictionary's keys. The values that get imputed are the values of the dictionary. If a value is not specified in the dictionary, the series' original value is retained.
</br></br>


```
Example:

 [IN]: column_a = pd.Series(['A', 'B', 'C', 'D'])
       column_a

[OUT]: 0    A
       1    B
       2    C
       3    D
       dtype: object

 [IN]: replacement_dict = {'A':'z', 'B':'y', 'C':'x'}
       column_a = column_a.replace(replacement_dict)
       column_a
       
[OUT]: 0    z
       1    y
       2    x
       3    D
       dtype: object

```

1. Create a dictionary called `replacement_dict` whose keys are the incorrect spellings in the `Industry` series and whose values are the correct spelling, as indicated in `industry_list`.

2. Call the `replace()` method on the `Industry` series and pass to it `replacement_dict` as its argument. Reassign the result back to the `Industry` column.

3. Verify that there are no longer any elements in `Industry` that are not in `industry_list`.

In [ ]:
# 1. Create `replacement_dict`

### YOUR CODE HERE ###

# 2. Replace the incorrect values in the `Industry` column

### YOUR CODE HERE ###

# 3. Verify that there are no longer any elements in `Industry` that are not in `industry_list`

### YOUR CODE HERE ###


#### Handling duplicate rows

The business mentioned that no company should appear in the data more than once.

Verify that this is indeed the case, and if not, clean the data so each company appears only once.

Begin by checking which, if any, companies are duplicated. Filter the data to return all occurrences of those duplicated companies.

In [ ]:
# Isolate rows of all companies that have duplicates

### YOUR CODE HERE ###


**Question: Do these duplicated companies seem like legitimate data points, or are they problematic data?**


[Write your response here. Double-click (or enter) to edit.]

Keep the first occurrence of each duplicate company and drop the subsequent rows that are copies.

In [ ]:
# Drop rows of duplicate companies after their first occurrence

### YOUR CODE HERE ###


**Question: Why is it important to perform input validation?**


[Write your response here. Double-click (or enter) to edit.]

**Question: What steps did you take to perform input validation for this dataset?**


[Write your response here. Double-click (or enter) to edit.]

### Convert numerical data to categorical data

Sometimes, you'll want to simplify a numeric column by converting it to a categorical column. To do this, one common approach is to break the range of possible values into a defined number of equally sized bins and assign each bin a name. In the next step, you'll practice this process.

#### Create a `High Valuation` column

The data in the `Valuation` column represents how much money (in billions, USD) each company is valued at. Use the `Valuation` column to create a new column called `High Valuation`. For each company, the value in this column should be `low` if the company is in the bottom 50% of company valuations and `high` if the company is in the top 50%.

In [ ]:
# Create new `High Valuation` column
# Use qcut to divide Valuation into 'high' and 'low' Valuation groups

### YOUR CODE HERE ###


### Convert categorical data to numerical data

Three common methods for changing categorical data to numerical are:

1. Label encoding: order matters (ordinal numeric labels)
2. Label encoding: order doesn't matter (nominal numeric labels)
3. Dummy encoding: order doesn't matter (creation of binary columns for each possible category contained in the variable)

The decision on which method to use depends on the context and must be made on a case-to-case basis. However, a distinction is typically made between categorical variables with equal weight given to all possible categories vs. variables with a hierarchical structure of importance to their possible categories.  

For example, a variable called `subject` might have possible values of `history`, `mathematics`, `literature`. In this case, each subject might be **nominal**&mdash;given the same level of importance. However, you might have another variable called `class`, whose possible values are `freshman`, `sophomore`, `junior`, `senior`. In this case, the class variable is **ordinal**&mdash;its values have an ordered, hierarchical structure of importance. 

Machine learning models typically need all data to be numeric, and they generally use ordinal label encoding (method 1) and dummy encoding (method 3). 

In the next steps, you'll convert the following variables: `Continent`, `Country/Region`, and `Industry`, each using a different approach.

### Convert `Continent` to numeric

For the purposes of this exercise, suppose that the investment group has specified that they want to give more weight to continents with fewer unicorn companies because they believe this could indicate unrealized market potential. 

**Question: Which type of variable would this make the `Continent` variable in terms of how it would be converted to a numeric data type?**


[Write your response here. Double-click (or enter) to edit.]

Rank the continents in descending order from the greatest number of unicorn companies to the least.

In [ ]:
# Rank the continents by number of unicorn companies

### YOUR CODE HERE ###

Now, create a new column called `Continent Number` that represents the `Continent` column converted to numeric in the order of their number of unicorn companies, where North America is encoded as `1`, through Africa, encoded as `6`.

In [ ]:
# Create numeric `Continent Number` column

### YOUR CODE HERE ###

### Convert `Country/Region` to numeric

Now, suppose that within a given continent, each company's `Country/Region` is given equal importance. For analytical purposes, you want to convert the values in this column to numeric without creating a large number of dummy columns. Use label encoding of this nominal categorical variable to create a new column called `Country/Region Numeric`, wherein each unique `Country/Region` is assigned its own number. 

In [ ]:
# Create `Country/Region Numeric` column
# Create numeric categories for Country/Region

### YOUR CODE HERE ###


### Convert `Industry` to numeric

Finally, create dummy variables for the values in the `Industry` column. 

In [ ]:
# Convert `Industry` to numeric data
# Create dummy variables with Industry values

### YOUR CODE HERE ###

# Combine `companies` DataFrame with new dummy Industry columns

### YOUR CODE HERE ###


Display the first few rows of `companies`

In [ ]:
### YOUR CODE HERE ###

**Question: Which categorical encoding approach did you use for each variable? Why?**

[Write your response here. Double-click (or enter) to edit.]

**Question: How does label encoding change the data?**


[Write your response here. Double-click (or enter) to edit.]

**Question: What are the benefits of label encoding?**


[Write your response here. Double-click (or enter) to edit.]

**Question: What are the disadvantages of label encoding?**


[Write your response here. Double-click (or enter) to edit.]

## Conclusion

**What are some key takeaways that you learned during this lab?**

[Write your response here. Double-click (or enter) to edit.]

**Reference**

[Bhat, M.A. *Unicorn Companies*](https://www.kaggle.com/datasets/mysarahmadbhat/unicorn-companies)

